In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# =========================================================
# 1. 데이터 준비 (The Problem)
# =========================================================
# XOR의 4가지 입력 케이스 (행렬 형태: 4행 2열)
# 기하학적 의미: 2차원 평면에 찍힌 4개의 점
X = np.array([
    [0,0], 
    [0,1], 
    [1,0], 
    [1,1]
], dtype=np.float32)

# XOR의 정답 레이블 (행렬 형태: 4행 1열)
# 0: 파란색 점, 1: 빨간색 점 (분류 대상)
y = np.array([
    [0], 
    [1], 
    [1], 
    [0]
], dtype=np.float32)

# =========================================================
# 2. 모델 설계 (The Architecture)
# =========================================================
# Sequential: "층(Layer)을 순서대로 쌓아 올리겠다"는 선언
model = Sequential()

# [은닉층 (Hidden Layer)] 
# 역할: 2차원 입력 공간을 휘고 구부려서(Warping) 선형 분리가 가능하게 만듦
# ---------------------------------------------------------
# units=2     : 뉴런 2개 사용. (기하학적 의미: 2개의 곡선/언덕을 만듦)
# input_dim=2 : 입력 데이터가 x1, x2 두 개임.
# activation='sigmoid' : 비선형 함수. (기하학적 의미: 평면을 S자로 부드럽게 휨)
# ---------------------------------------------------------
# 수식: H = sigmoid(X @ W1 + b1)
model.add(Dense(units=2, input_dim=2, activation='sigmoid'))

# [출력층 (Output Layer)]
# 역할: 휘어진 공간에서 최종적으로 영역을 잘라내어 0 또는 1을 판단
# ---------------------------------------------------------
# units=1     : 최종 출력은 0~1 사이의 숫자 1개.
# activation='sigmoid' : 결과를 확률값(0.0 ~ 1.0)으로 변환.
# ---------------------------------------------------------
# 수식: Y_pred = sigmoid(H @ W2 + b2)
model.add(Dense(units=1, activation='sigmoid'))


# =========================================================
# 3. 학습 설정 (The Strategy)
# =========================================================
# Optimizer (Adam): 경사 하강법(Gradient Descent)을 똑똑하게 수행하는 도구
# learning_rate=0.01: 한 번 업데이트할 때 이동할 보폭.
# (기본값 0.001은 너무 작아서 0.5의 늪(Local Minima)에 빠질 수 있어 10배 키움)
optimizer = Adam(learning_rate=0.01)

# Compile: 모델 조립
# loss='binary_crossentropy': 이진 분류용 채점 기준 (틀리면 로그 함수로 큰 벌점)
# metrics=['accuracy']: 우리가 보기 편하게 '정확도'도 같이 기록해라
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])


# =========================================================
# 4. 학습 수행 (The Learning)
# =========================================================
# epochs=1000: 문제집(데이터 4개)을 1000번 반복해서 풀어라.
# verbose=0: 학습 과정을 화면에 지저분하게 출력하지 마라.
# 내부 동작: 
#   1) 순전파(Forward)로 예측
#   2) 오차(Loss) 계산
#   3) 역전파(Backward)로 기울기 계산
#   4) 가중치 업데이트 (W <- W - lr * Gradient)
print("학습 시작... (약 1000 Epoch)")
history = model.fit(X, y, epochs=1000, verbose=0)
print("학습 완료!")


# =========================================================
# 5. 결과 검증 (The Verification)
# =========================================================
pred = model.predict(X)

print("\n[최종 예측 결과 (확률값)]")
print(pred) # 예: [[0.04], [0.97], [0.96], [0.03]] 처럼 나옴

print("\n[정답으로 변환 (반올림)]")
# 0.5 이상이면 1, 미만이면 0으로 변환
print(np.round(pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Keras 예측 결과:
 [[0.02846328]
 [0.96662706]
 [0.97150636]
 [0.0256856 ]]
[[0.]
 [1.]
 [1.]
 [0.]]


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
import numpy as np

# 1. 데이터 준비 (XOR)
# TensorFlow는 float32 타입을 기본으로 사용합니다.
X = tf.constant([[0,0], [0,1], [1,0], [1,1]], dtype=tf.float32)
y = tf.constant([[0],   [1],   [1],   [0]],   dtype=tf.float32)

# 2. 모델 요소 정의 (Layer를 낱개로 준비)
# 발표용 시각화를 위해 activation을 층 안에 넣지 않고 따로 뺐습니다.
dense1 = Dense(units=2, activation=None, name="Hidden_Layer_Linear") # 선형
dense2 = Dense(units=1, activation=None, name="Output_Layer_Linear") # 선형

# 가중치 초기화를 위해 한 번 통과시킴 (Build)
dense1(X)
dense2(dense1(X))

# 학습률 설정 (0.1)
optimizer = Adam(learning_rate=0.1)
loss_fn = BinaryCrossentropy()

print(f"{'='*20} [학습 시작 전 가중치] {'='*20}")
# kernel: 가중치(W), bias: 편향(b)
print(f"Layer 1 Weights (W1):\n{dense1.kernel.numpy()}")
print(f"Layer 2 Weights (W2):\n{dense2.kernel.numpy()}\n")


# =================================================================
# ★ Keras의 블랙박스(model.fit)를 해제하는 핵심 도구: GradientTape
# =================================================================

print(f"{'='*20} [1. 순전파 (Forward Pass)] {'='*20}")

with tf.GradientTape() as tape:
    # Tape 안에서 벌어지는 모든 연산은 녹화됩니다 (미분을 위해)
    
    # Step 1: 첫 번째 선형 변환 (Z1 = W1*X + b1)
    z1 = dense1(X)
    print(f"▶ Step 1 [Linear 1]: 입력 공간 변환 (z1)\n{z1.numpy()}\n")
    
    # Step 2: 활성화 함수 (H1 = Sigmoid(Z1))
    h1 = tf.nn.sigmoid(z1)
    print(f"▶ Step 2 [Activation]: 공간을 휘게 만듦 (h1)\n{h1.numpy()}\n")
    
    # Step 3: 두 번째 선형 변환 (Z2 = W2*H1 + b2)
    z2 = dense2(h1)
    print(f"▶ Step 3 [Linear 2]: 영역 합치기/자르기 (z2)\n{z2.numpy()}\n")
    
    # Step 4: 최종 출력 (Y_pred = Sigmoid(Z2))
    y_pred = tf.nn.sigmoid(z2)
    print(f"▶ Step 4 [Output]: 최종 예측 확률 (y_pred)\n{y_pred.numpy()}\n")
    
    # Step 5: 오차 계산 (Loss)
    loss = loss_fn(y, y_pred)
    print(f"{'='*20} [2. 오차 계산 (Loss)] {'='*20}")
    print(f"Loss 값: {loss.numpy():.4f}")
    print("(이 오차를 줄이기 위해 테이프를 되감습니다!)\n")


print(f"{'='*20} [3. 역전파 (Backward Pass)] {'='*20}")

# 학습 가능한 변수들 모으기 (W1, b1, W2, b2)
trainable_vars = dense1.trainable_variables + dense2.trainable_variables

# ★ 테이프를 이용해서 미분(Gradient) 계산!
# grads 리스트에는 [dL/dW1, dL/db1, dL/dW2, dL/db2] 순서로 들어감
grads = tape.gradient(loss, trainable_vars)

print("GradientTape가 연산 과정을 역추적하여 기울기를 구했습니다.\n")

# Keras는 변수 리스트 순서대로 기울기가 나옵니다.
# dense1.trainable_variables -> [kernel(W), bias(b)]
print(f"◀ Gradient @ Layer 2 Weights (dL/dW2): 출력층 가중치 수정 방향")
print(f"{grads[2].numpy()}\n") # W2의 기울기

print(f"◀ Gradient @ Layer 1 Weights (dL/dW1): 은닉층 가중치 수정 방향")
print(f"{grads[0].numpy()}\n") # W1의 기울기


print(f"{'='*20} [4. 파라미터 업데이트 (Update)] {'='*20}")

# 옵티마이저가 기울기를 보고 가중치를 수정함
# zip(gradients, variables)로 짝을 지어서 전달
optimizer.apply_gradients(zip(grads, trainable_vars))

print("Optimizer가 학습률(0.1)을 적용해 가중치를 업데이트했습니다.\n")
print(f"Updated Layer 1 Weights:\n{dense1.kernel.numpy()}")
print(f"Updated Layer 2 Weights:\n{dense2.kernel.numpy()}")

==================== [학습 시작 전 가중치] ====================
Layer 1 Weights (W1):
[[ 0.2538452   0.7059097 ]
 [-0.6374692   0.51765466]]
Layer 2 Weights (W2):
[[1.1513721 ]
 [0.68751395]]

==================== [1. 순전파 (Forward Pass)] ====================
▶ Step 1 [Linear 1]: 입력 공간 변환 (z1)
[[ 0.          0.        ]
 [-0.6374692   0.51765466]
 [ 0.2538452   0.7059097 ]
 [-0.38362396  1.2235644 ]]

▶ Step 2 [Activation]: 공간을 휘게 만듦 (h1)
[[0.5       0.5      ]
 [0.3458189 0.6265992]
 [0.5631227 0.6694967]
 [0.4052531 0.7726902]]

▶ Step 3 [Linear 2]: 영역 합치기/자르기 (z2)
[[0.919443 ]
 [0.8289619]
 [1.1086521]
 [0.9978324]]

▶ Step 4 [Output]: 최종 예측 확률 (y_pred)
[[0.71492857]
 [0.6961354 ]
 [0.7518777 ]
 [0.7306322 ]]

==================== [2. 오차 계산 (Loss)] ====================
Loss 값: 0.8035
(이 오차를 줄이기 위해 테이프를 되감습니다!)

==================== [3. 역전파 (Backward Pass)] ====================
GradientTape가 연산 과정을 역추적하여 기울기를 구했습니다.

◀ Gradient @ Layer 2 Weights (dL/dW2): 출력층 가중치 수정 방향
[[0.10218746]
 [0.14137